# DX 704 Week 1 Project

This week's project will build a portfolio risk and return model, and make investing recommendations for hypothetical clients.
You will collect historical data, estimate returns and risks, construct efficient frontier portfolios, and sanity check the certainty of the maximum return portfolio.

The full project description and a template notebook are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-01


Feel free to use optimization tools or libraries (such as CVXOPT or scipy.optimize) to perform any calculations required for this mini project.

### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Data

Collect historical monthly price data for the last 24 months covering 6 different stocks.
The data should cover 24 consecutive months including the last month that ended before this week's material was released on Blackboard.
To be clear, if a month ends between the Blackboard release and submitting your project, you do not need to add that month.

The six different stocks must include AAPL, SPY and TSLA.
At least one of the remaining 3 tickers must start with the same letter as your last name (e.g. professor Considine could use COIN).
This is to encourage diversity in what stocks you analyze; if you discuss this project with classmates, please make sure that you pick different tickers to differentiate your work.
Do not pick stocks with fewer than 24 consecutive months of price data.

In [54]:
# YOUR CHANGES HERE
import yfinance as yf
import pandas as pd

# 1. Pick your tickers
tickers = ["AAPL", "SPY", "TSLA", "DIS", "GOOG", "MSFT"]

# 2. Download 2 years of monthly data
# interval="1mo" → monthly data
data = yf.download(tickers, start="2023-09-01", end="2025-09-01", interval="1mo")['Close']

data.index = data.index + pd.offsets.MonthEnd(0)

# 4. Show the data
print(data.tail(24))  # last 24 months


/tmp/ipython-input-3790959639.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start="2023-09-01", end="2025-09-01", interval="1mo")['Close']
[*********************100%***********************]  6 of 6 completed

Ticker            AAPL         DIS        GOOG        MSFT         SPY  \
Date                                                                     
2023-09-30  169.549271   79.721001  131.070892  311.062317  416.396851   
2023-10-31  169.113525   80.252136  124.559601  333.090363  408.794403   
2023-11-30  188.107513   91.170135  133.128662  373.284637  446.135193   
2023-12-31  190.913681   88.809486  140.097229  371.209137  464.619873   
2024-01-31  182.851913   94.782936  140.962112  392.472412  473.933441   
2024-02-29  179.232574  110.108070  138.954041  408.326111  498.666504   
2024-03-31  170.257355  120.745865  151.360291  416.083130  513.378845   
2024-04-30  169.115540  109.634399  163.667130  385.039062  494.210358   
2024-05-31  190.879272  102.539261  172.932068  410.554779  519.207275   
2024-06-30  209.401932   97.980194  182.336166  442.821381  535.796509   
2024-07-31  220.795654   92.454071  172.322708  414.485565  544.033997   
2024-08-31  227.675644   89.599197  16

Save the data as a TSV file named "historical_prices.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
The date should be the last trading day of the month, so it may not be the last day of the month.
For example, the last trading day of November 2024 was 2024-11-29.
The remaining columns should contain the adjusted closing prices of the corresponding stock tickers on that day.


In [55]:
# YOUR CHANGES HERE
# 3. Reset index so "Date" becomes a column
data = data.reset_index()

# 4. Rename the Date column to "date"
data = data.rename(columns={"Date": "date"})

# 5. Save as TSV (Tab-Separated Values)
data.to_csv("historical_prices.tsv", sep="\t", index=False)

print("Saved historical_prices.tsv")


Saved historical_prices.tsv


In [80]:
from google.colab import files
files.download("historical_prices.tsv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [56]:
print(data.to_string())

Ticker       date        AAPL         DIS        GOOG        MSFT         SPY        TSLA
0      2023-09-30  169.549271   79.721001  131.070892  311.062317  416.396851  250.220001
1      2023-10-31  169.113525   80.252136  124.559601  333.090363  408.794403  200.839996
2      2023-11-30  188.107513   91.170135  133.128662  373.284637  446.135193  240.080002
3      2023-12-31  190.913681   88.809486  140.097229  371.209137  464.619873  248.479996
4      2024-01-31  182.851913   94.782936  140.962112  392.472412  473.933441  187.289993
5      2024-02-29  179.232574  110.108070  138.954041  408.326111  498.666504  201.880005
6      2024-03-31  170.257355  120.745865  151.360291  416.083130  513.378845  175.789993
7      2024-04-30  169.115540  109.634399  163.667130  385.039062  494.210358  183.279999
8      2024-05-31  190.879272  102.539261  172.932068  410.554779  519.207275  178.080002
9      2024-06-30  209.401932   97.980194  182.336166  442.821381  535.796509  197.880005
10     202

Submit "historical_prices.tsv" in Gradescope.

## Part 2: Calculate Historical Asset Returns

Calculate the historical asset returns based on the price data that you previously collected.

In [57]:
# YOUR CHANGES HERE


# Read your price data
data = pd.read_csv("historical_prices.tsv", sep="\t")

# Set date as index
data = data.set_index("date")

# Calculate monthly returns for each stock
returns = data.pct_change()   # This calculates (current/previous - 1) for each column

# Drop the first row (it will be NaN because there's no previous month to compare)
returns = returns.dropna()

print(returns)



                AAPL       DIS      GOOG      MSFT       SPY      TSLA
date                                                                  
2023-10-31 -0.002570  0.006662 -0.049678  0.070816 -0.018258 -0.197346
2023-11-30  0.112315  0.136046  0.068795  0.120671  0.091344  0.195379
2023-12-31  0.014918 -0.025893  0.052345 -0.005560  0.041433  0.034988
2024-01-31 -0.042227  0.067261  0.006173  0.057281  0.020046 -0.246257
2024-02-29 -0.019794  0.161687 -0.014245  0.040394  0.052187  0.077901
2024-03-31 -0.050076  0.096612  0.089283  0.018997  0.029503 -0.129235
2024-04-30 -0.006706 -0.092024  0.081308 -0.074610 -0.037338  0.042608
2024-05-31  0.128692 -0.064716  0.056608  0.066268  0.050580 -0.028372
2024-06-30  0.097039 -0.044462  0.054380  0.078593  0.031951  0.111186
2024-07-31  0.054411 -0.056400 -0.054918 -0.063989  0.015374  0.172781
2024-08-31  0.031160 -0.030879 -0.046434 -0.002892  0.023365 -0.077390
2024-09-30  0.018645  0.064284  0.012598  0.033407  0.017882  0.221942
2024-1

Save the data as a TSV file named "historical_returns.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
Each row should have the date at the end of the month and the corresponding *relative* price changes.
For example, if the previous price was \$100 and the new price is \$110, the return value should be 0.10.
There should only be 23 rows of data in this file, since they are computed as the differences of 24 prices.

In [58]:
# YOUR CHANGES HERE

# Save the returns DataFrame to a TSV file
returns.to_csv("historical_returns.tsv", sep="\t", index_label="date")


In [59]:
print(returns.to_string())

                AAPL       DIS      GOOG      MSFT       SPY      TSLA
date                                                                  
2023-10-31 -0.002570  0.006662 -0.049678  0.070816 -0.018258 -0.197346
2023-11-30  0.112315  0.136046  0.068795  0.120671  0.091344  0.195379
2023-12-31  0.014918 -0.025893  0.052345 -0.005560  0.041433  0.034988
2024-01-31 -0.042227  0.067261  0.006173  0.057281  0.020046 -0.246257
2024-02-29 -0.019794  0.161687 -0.014245  0.040394  0.052187  0.077901
2024-03-31 -0.050076  0.096612  0.089283  0.018997  0.029503 -0.129235
2024-04-30 -0.006706 -0.092024  0.081308 -0.074610 -0.037338  0.042608
2024-05-31  0.128692 -0.064716  0.056608  0.066268  0.050580 -0.028372
2024-06-30  0.097039 -0.044462  0.054380  0.078593  0.031951  0.111186
2024-07-31  0.054411 -0.056400 -0.054918 -0.063989  0.015374  0.172781
2024-08-31  0.031160 -0.030879 -0.046434 -0.002892  0.023365 -0.077390
2024-09-30  0.018645  0.064284  0.012598  0.033407  0.017882  0.221942
2024-1

In [81]:

files.download("historical_returns.tsv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "historical_returns.tsv" in Gradescope.

## Part 3: Estimate Returns

Estimate the expected returns for each asset using the previously calculated return data.
Just compute the average (mean) return for each asset over your data set; do not use other estimators that have been mentioned.
This will serve as your estimate of expected return for each asset.

In [60]:
# YOUR CHANGES HERE

# Compute the average return for each stock
expected_returns = returns.mean()

# Show the result
print(expected_returns)


AAPL    0.015376
DIS     0.021776
GOOG    0.023812
MSFT    0.023229
SPY     0.019807
TSLA    0.025313
dtype: float64


Save the estimated returns in a TSV file named "estimated_returns.tsv" and include a header row with the column names "asset" and "estimated_return".

In [61]:
# YOUR CHANGES HERE

# 1. Convert expected returns (Series) into a DataFrame
df_expected = expected_returns.reset_index()

# 2. Rename the columns
df_expected.columns = ["asset", "estimated_return"]

# 3. Save to TSV (tab-separated file)
df_expected.to_csv("estimated_returns.tsv", sep="\t", index=False)

print("Saved estimated_returns.tsv")

Saved estimated_returns.tsv


In [82]:
files.download("estimated_returns.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "estimated_returns.tsv" in Gradescope.

In [62]:
print(df_expected.to_string())

  asset  estimated_return
0  AAPL          0.015376
1   DIS          0.021776
2  GOOG          0.023812
3  MSFT          0.023229
4   SPY          0.019807
5  TSLA          0.025313


## Part 4: Estimate Risk

Estimate the covariance matrix for the asset returns to understand how the assets move together.

In [63]:
# YOUR CHANGES HERE
# 1. Read your returns file
returns = pd.read_csv("historical_returns.tsv", sep="\t")

# 2. Set the date as index
returns = returns.set_index("date")

# 3. Calculate the covariance matrix
cov_matrix = returns.cov()

# 4. Show the result
print(cov_matrix)


          AAPL       DIS      GOOG      MSFT       SPY      TSLA
AAPL  0.003591 -0.000430  0.000736  0.000353  0.000785  0.003234
DIS  -0.000430  0.009851  0.000572  0.003634  0.002477  0.005682
GOOG  0.000736  0.000572  0.004830  0.001105  0.000747  0.004309
MSFT  0.000353  0.003634  0.001105  0.004005  0.001450  0.002180
SPY   0.000785  0.002477  0.000747  0.001450  0.001259  0.002339
TSLA  0.003234  0.005682  0.004309  0.002180  0.002339  0.026703


Save the estimated covariances to a TSV file named "estimated_covariance.tsv".
The header row should have a blank column name followed by the names of the assets.
Each data row should start with the name of an asset for that row, and be followed by the individual covariances corresponding to that row and column's assets.
(This is the format of pandas's `to_csv` method with `sep="\t"` when used on a covariance matrix as computed in the examples.)

In [64]:
# YOUR CHANGES HERE

# Read historical returns
returns = pd.read_csv("historical_returns.tsv", sep="\t", index_col="date")

# Calculate covariance matrix
cov_matrix = returns.cov()

# Save as TSV
cov_matrix.to_csv("estimated_covariance.tsv", sep="\t")

print("Saved estimated_covariance.tsv")

Saved estimated_covariance.tsv


In [83]:

files.download("estimated_covariance.tsv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "estimated_covariance.tsv" in Gradescope.

## Part 5: Construct the Maximum Return Portfolio

Compute the maximum return portfolio based on your previously estimated risks and returns.

In [65]:
# YOUR CHANGES HERE

# Read your estimated returns
df_returns = pd.read_csv("estimated_returns.tsv", sep="\t")

# Find the asset with the maximum return
max_return_asset = df_returns.loc[df_returns['estimated_return'].idxmax()]

print("Maximum return asset:", max_return_asset['asset'])
print("Expected monthly return:", max_return_asset['estimated_return'])

Maximum return asset: TSLA
Expected monthly return: 0.0253129037321271


Save the maximum return portfolio in a TSV file named "maximum_return.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [66]:
# YOUR CHANGES HERE
# Find the asset with the maximum return
max_asset = df_returns.loc[df_returns['estimated_return'].idxmax(), 'asset']

# Create the portfolio allocations: 1 for the max asset, 0 for others
df_allocation = pd.DataFrame({
    'asset': df_returns['asset'],
    'allocation': df_returns['asset'].apply(lambda x: 1.0 if x == max_asset else 0.0)
})

# Save to TSV
df_allocation.to_csv("maximum_return.tsv", sep="\t", index=False)

print("Saved maximum_return.tsv")


Saved maximum_return.tsv


In [67]:
print(df_allocation.to_string())

  asset  allocation
0  AAPL         0.0
1   DIS         0.0
2  GOOG         0.0
3  MSFT         0.0
4   SPY         0.0
5  TSLA         1.0


In [84]:
files.download("maximum_return.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "maximum_return.tsv" in Gradescope.

## Part 6: Construct the Minimum Risk Portfolio

Compute the minimum return portfolio based on your previously estimated risks and returns.

In [68]:
# YOUR CHANGES HERE
import numpy as np
from scipy.optimize import minimize

# Read historical returns
returns = pd.read_csv("historical_returns.tsv", sep="\t", index_col="date")

# Calculate covariance matrix
cov_matrix = returns.cov()

# Number of assets
n_assets = len(cov_matrix)

# Function to compute portfolio variance
def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix.values @ weights

# Constraints: sum of weights = 1
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

# Bounds for weights: between 0 and 1
bounds = tuple((0, 1) for _ in range(n_assets))

# Initial guess (equal allocation)
init_guess = np.array([1/n_assets] * n_assets)

# Minimize variance
result = minimize(portfolio_variance, init_guess,
                  args=(cov_matrix,), method='SLSQP',
                  bounds=bounds, constraints=constraints)

# Get the optimal weights
min_risk_weights = result.x

# Save as DataFrame
df_min_risk = pd.DataFrame({
    'asset': cov_matrix.columns,
    'allocation': min_risk_weights
})

# Save to TSV
df_min_risk.to_csv("minimum_risk.tsv", sep="\t", index=False)

print("Saved minimum_risk.tsv")
print(df_min_risk)

Saved minimum_risk.tsv
  asset  allocation
0  AAPL    0.242673
1   DIS    0.000000
2  GOOG    0.039343
3  MSFT    0.121026
4   SPY    0.596957
5  TSLA    0.000000


Save the minimum risk portfolio in a TSV file named "minimum_risk.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [69]:
# YOUR CHANGES HERE

# List of asset names (same order as your covariance matrix)
assets = cov_matrix.columns

# Create a DataFrame with asset names and allocations
df_min_risk = pd.DataFrame({
    'asset': assets,
    'allocation': min_risk_weights
})

# Save the DataFrame to a TSV file
df_min_risk.to_csv("minimum_risk.tsv", sep="\t", index=False)

print("Saved minimum_risk.tsv")
print(df_min_risk)

Saved minimum_risk.tsv
  asset  allocation
0  AAPL    0.242673
1   DIS    0.000000
2  GOOG    0.039343
3  MSFT    0.121026
4   SPY    0.596957
5  TSLA    0.000000


In [85]:
files.download("minimum_risk.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "minimum_risk.tsv" in Gradescope.

## Part 7: Build Efficient Frontier Portfolios

Compute 101 portfolios along the mean-variance efficient frontier with evenly spaced estimated returns.
The first portfolio should be the minimum risk portfolio from part 4, and the last portfolio should be the maximum return portfolio from part 3.
The estimated return of each portfolio should be higher than the previous by one percent of the difference between the first and last portfolios.
That is, the estimated return of the portfolios should be similar to `np.linspace(min_risk_return, max_return, 101)`.


In [70]:
import pandas as pd
max_return_df = pd.read_csv("maximum_return.tsv", sep="\t")
max_return_weights = max_return_df['allocation'].values


In [71]:
min_risk_df = pd.read_csv("minimum_risk.tsv", sep="\t")
min_risk_weights = min_risk_df['allocation'].values


In [72]:
expected_returns = pd.read_csv("estimated_returns.tsv", sep="\t")
expected_returns = expected_returns.set_index("asset")['estimated_return']


In [73]:
cov_matrix = pd.read_csv("estimated_covariance.tsv", sep="\t", index_col=0)


In [74]:
min_risk_return = np.dot(min_risk_weights, expected_returns.values)
max_return = np.dot(max_return_weights, expected_returns.values)

print("Minimum risk portfolio return:", min_risk_return)
print("Maximum return portfolio return:", max_return)

Minimum risk portfolio return: 0.01930372861057142
Maximum return portfolio return: 0.0253129037321271


In [75]:
target_returns = np.linspace(min_risk_return, max_return, 101)


In [76]:
efficient_weights = []

for r in target_returns:
    # alpha goes from 0 (min risk) to 1 (max return)
    alpha = (r - min_risk_return) / (max_return - min_risk_return)
    weights = (1 - alpha) * min_risk_weights + alpha * max_return_weights
    efficient_weights.append(weights)

efficient_weights = np.array(efficient_weights)  # 101 x 6 array


Save the portfolios in a TSV file named "efficient_frontier.tsv".
The header row should have columns "index", "return", "risk", and all the asset tickers.
Each data row should have the portfolio index (0-100), the estimated return of the portfolio, the estimated standard deviation (not variance) of the portfolio, and all the asset allocations (which should sum to one).

In [77]:
# YOUR CHANGES HERE

# ------------------------------
# 1️⃣ Read the previously saved files
# ------------------------------
# Estimated returns
df_returns = pd.read_csv("estimated_returns.tsv", sep="\t")
assets = df_returns['asset'].tolist()
expected_returns = df_returns['estimated_return'].values  # array

# Estimated covariance matrix
cov_matrix = pd.read_csv("estimated_covariance.tsv", sep="\t", index_col=0)
cov_matrix = cov_matrix[assets]  # Ensure order matches

# ------------------------------
# 2️⃣ Construct maximum return and minimum risk portfolios
# ------------------------------
# Maximum return portfolio: invest 100% in asset with max return
max_idx = df_returns['estimated_return'].idxmax()
max_return_weights = np.array([1.0 if i == max_idx else 0.0 for i in range(len(assets))])

# Minimum risk portfolio: invest 100% in asset with lowest variance (diagonal of covariance)
min_idx = np.argmin(np.diag(cov_matrix.values))
min_risk_weights = np.array([1.0 if i == min_idx else 0.0 for i in range(len(assets))])

# ------------------------------
# 3️⃣ Compute min and max portfolio returns
# ------------------------------
min_risk_return = np.dot(min_risk_weights, expected_returns)
max_return = np.dot(max_return_weights, expected_returns)

# ------------------------------
# 4️⃣ Generate 101 portfolios along the efficient frontier
# ------------------------------
target_returns = np.linspace(min_risk_return, max_return, 101)
efficient_weights = []

for r in target_returns:
    alpha = (r - min_risk_return) / (max_return - min_risk_return)
    weights = (1 - alpha) * min_risk_weights + alpha * max_return_weights
    efficient_weights.append(weights)

efficient_weights = np.array(efficient_weights)

# ------------------------------
# 5️⃣ Compute portfolio risk (standard deviation)
# ------------------------------
portfolio_risks = []
for w in efficient_weights:
    risk = np.sqrt(np.dot(w.T, np.dot(cov_matrix.values, w)))
    portfolio_risks.append(risk)

portfolio_risks = np.array(portfolio_risks)

# ------------------------------
# 6️⃣ Create DataFrame and save to TSV
# ------------------------------
ef_df = pd.DataFrame(efficient_weights, columns=assets)
ef_df.insert(0, "risk", portfolio_risks)
ef_df.insert(0, "return", target_returns)
ef_df.insert(0, "index", range(101))  # 0–100 portfolio index

# Save to TSV
ef_df.to_csv("efficient_frontier.tsv", sep="\t", index=False)
print("Saved efficient_frontier.tsv")
...

Saved efficient_frontier.tsv


Ellipsis

In [86]:
files.download("efficient_frontier.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "efficient_frontier.tsv" in Gradescope.

## Part 8: Check Maximum Return Portfolio Stability

Check the stability of the maximum return portfolio by resampling the estimated risk/return model.

Repeat 1000 times -
1. Use `np.random.multivariate_normal` to generate 23 return samples using your previously estimated risks and returns.
2. Estimate the return of each asset using that resampled return history.
3. Check which asset had the highest return in those resampled estimates.

This procedure is a reduced and simplified version of the Michaud resampled efficient frontier procedure that takes uncertainty in the risk model into account.

In [78]:
# YOUR CHANGES HERE

# Set up simulation
num_simulations = 1000
num_months = 23
max_return_counts = {asset: 0 for asset in assets}

# Run simulations
for i in range(num_simulations):
    # Generate 23 samples of returns for all assets
    simulated_returns = np.random.multivariate_normal(mean=expected_returns,
                                                      cov=cov_matrix.values,
                                                      size=num_months)

    # Compute average return of each asset in this simulation
    avg_returns = simulated_returns.mean(axis=0)

    # Find which asset had maximum return
    max_idx = np.argmax(avg_returns)
    max_return_counts[assets[max_idx]] += 1

# Show results
print("Maximum return asset frequency over 1000 resamples:")
for asset, count in max_return_counts.items():
    print(f"{asset}: {count} times")


Maximum return asset frequency over 1000 resamples:
AAPL: 77 times
DIS: 195 times
GOOG: 211 times
MSFT: 150 times
SPY: 12 times
TSLA: 355 times


Save a file "max_return_probabilities.tsv" with the distribution of highest return assets.
The header row should have columns "asset" and "probability".
There should be a data row for each asset and its sample probability of having the highest return based on those 1000 resampled estimates.


In [79]:
# YOUR CHANGES HERE
# Convert counts to probabilities
total_simulations = num_simulations
results = []

for asset, count in max_return_counts.items():
    prob = count / total_simulations
    results.append({'asset': asset, 'probability': prob})

# Create DataFrame
df_probabilities = pd.DataFrame(results)

# Save to TSV
df_probabilities.to_csv("max_return_probabilities.tsv", sep="\t", index=False)

print("Saved max_return_probabilities.tsv")



Saved max_return_probabilities.tsv


In [87]:
files.download("max_return_probabilities.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "max_return_probabilities.tsv" in Gradescope.

## Part 9: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgements.txt" in Gradescope.

## Part 10: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.